In [1]:
## CENTERING SLAB (FROM VASP FILE) AND GENERATING THE QE INPUT FILE ##

In [1]:
import ase
from ase.build import bulk
from ase.io import read,write,cif,vasp,espresso
from ase.io.espresso import write_espresso_in
from ase import Atoms
from ase.visualize import view

In [4]:
# Converting the slab from .vasp file into an 'atoms' object with ase
vasp_file = 'CaB6_1x4_slab.vasp'
slab = ase.io.vasp.read_vasp(vasp_file)
view(slab, viewer="x3d")

#note: if you wanto to change the vacuum width, edit the .vasp file

In [5]:
cell_parameters = slab.get_cell() # Get cell parameters
z_dimension = cell_parameters[2][2] # Get cell parameter in z-direction (the height of the cell)
original_positions = slab.get_positions() # Get the coordinates of all atoms
centered_slab_positions = original_positions.copy() # Copy the array with the original coordinates of all atoms in a new array to further modify it

In [6]:
# Recover the z-axis coordinates of all atoms to get the highest value
z_coordinates_list = []
for position in original_positions:
    coordinate_in_z = position[2]
    z_coordinates_list.append(coordinate_in_z)

max_z_value = max(z_coordinates_list)

In [7]:
# Calculating some necessary variables

half_max_z_value = max_z_value/2 # Divide the highest value in z-direction by 2
half_z_dimension = z_dimension/2 # Divide the height of the cell by 2
D = half_max_z_value + half_z_dimension

In [8]:
# Creating new (centered) coordinates displacing all atoms by 'D' along the z-direction
for position in centered_slab_positions:
    displacement = D - position[2]
    position[2] = displacement

In [9]:
# Centering the slab by setting the modified coordinates
slab.set_positions(centered_slab_positions)
view(slab, viewer="x3d")

In [10]:
## GENERATING QE INPUT FILE (TEMPLATE) WITH THE NEW COORDINATES OF THE CENTERED SLAB

# Fake (or real if desired) Pseudopotentials
pseudopotentials={'B': 'B.pbe-rrkjus.UPF', 'Ca': 'Ca.pbe-rrkjus.UPF'}

# Fake input data
input_data = {
    'calculation': 'relax',
    'restart_mode': 'from_scratch',
    'tprnfor': True,
    'etot_conv_thr': 1e-5,
    'forc_conv_thr': 1e-4,
    'ecutwfc': 60,
    'ecutrho': 480,
    'input_dft': 'rpbe',
    'vdw_corr': 'dft-d3',
    'occupations': 'smearing',
    'degauss': 0.01,
    'smearing': 'cold',
    'conv_thr': 1e-8,
    'mixing_mode': 'local-TF',
    'mixing_beta': 0.35,
    'diagonalization': 'david',
    'ion_dynamics': 'bfgs',
    'bfgs_ndim': 6,
    'startingwfc': 'random',
}  # This flat dictionary will be converted to a nested dictionary where, for example, "calculation" will be put into the "control" section

pw_input_template = 'pw_template.in'
write(pw_input_template, slab, input_data=input_data, pseudopotentials=pseudopotentials, format='espresso-in')